# Foundations of Data Science
## Semester 2 Week 05: Getting data and web scraping

**Learning outcomes:** In this lab you will learn about getting data via web scraping. By the end of this lab you should be able to:

- Identify when you need to use web scraping to retrieve data
- Determine whether a proposed web scraping exercise is legal
- Use the Python `requests` module to retrieve a web page in a responsible fashion
- Use the Python `BeautifulSoup` package to isolate and retrieve data from a HTML web page
- Understand how web scraping can be automated

This lab is somewhat inspired by this one 
https://ourcodingclub.github.io/tutorials/webscraping/ .

This lab is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/).

You might wish to review the contents of the S1 week 4 lecture before or after this lab.

In [ ]:
import re
import pandas as pd
import requests
# Package to display the hints and soultions
from common.show_solutions import show

## A. When do we need to do web scraping to retrieve data?

Web scraping is the process of automating the process of extracting data from a web page.

Spring is in the air (in Edinburgh at least), so we might be inspired to find out more about the birds around us. We'll imagine you want to collect information on the characteristics of UK birds, and put them in a table like this:

| Species            | Length   | Wingspan    | Weight                         | UK passage   | Europe          | UK breeding   |
|:-------------------|:---------|:------------|:-------------------------------|:-------------|:----------------|:--------------|
| Aquatic warbler    | 13cm     | 16.5-19.5cm | 10-14g                         | 42 birds     | 12-20,000 pairs | nan           |
| White-tailed eagle | 70-90cm  | 200-240cm   | 3.5-5kg (male); 4-7kg (female) | nan          | nan             | 106 pairs     |


It’s easy enough to head to the [Royal Society for the Protection of Birds (RSPB) A-Z listing of birds](https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/), click through each page, then copy the relevant information and paste it into a spreadsheet. Now imagine you want to repeat this for all 238 birds! This can quickly become very tedious as you click between lots of pages, repeating the same actions over and over again. It also increases the chance of making mistakes when copying and pasting. By automating this process, i.e. web scraping, you can reduce the chance of making mistakes and speed up your data collection. Additionally, once you have written the script, it can be adapted for a lot of different projects, saving time in the long run.

The first rule of web scraping is **don't do it unless you have to**. Data is increasingly available in structured formats such as CSV and JSON. Web scraping seems cool, but it's also time-consuming and fiddly. It pays to spend some time searching for files in structured formats, in which the data may well be in a cleaner format than on a web page. However, if you can't find the data you need in a structured format, but it is available on a web page, then web scraping can be helpful.

## B. Before we start - the law and ethics of web-scraping

It is worth remembering that web scraping involves a relationship between you, the web scraper, and the owner of the website, who may have invested considerable time and money in creating their site. It's possible that by web scraping you could have an adverse effect on either their intellectual property or their web server, so we need to think a little about law and ethics.

The law around web scraping is not always clear ([Byrce Davies - _Is web scraping legal?_](https://scrapediary.com/is-web-scraping-legal/)) but you should always check the terms and conditions of the website before before starting scraping. For example, [the Copyright Policy of the Financial Times website](https://help.ft.com/legal-privacy/copyright-policy/) says you cannot
> "Frame, harvest or scrape FT content or otherwise access FT content for similar purposes."

Beyond the legal restrictions for a specific website, we should also consider general ethical principles of web scraping. James Densmore, in his article [*Ethics in webscraping*](https://towardsdatascience.com/ethics-in-web-scraping-b96b18136f01), suggests the following set rules for those undertaking web scraping:

> - "If you have a public API that provides the data I’m looking for, I’ll use it and avoid scraping all together.
> - I will always provide a User Agent<sup>*</sup> string that makes my intentions clear and provides a way for you to contact me with questions or concerns.
> - I will request data at a reasonable rate. I will strive to never be confused for a [DDoS](https://en.wikipedia.org/wiki/Denial-of-service_attack) attack.
> - I will only save the data I absolutely need from your page. [...]
> - I will respect any content I do keep. I’ll never pass it off as my own.
> - I will look for ways to return value to you. Maybe I can drive some (real) traffic to your site or credit you in an article or post.
> - I will respond in a timely fashion to your outreach and work with you towards a resolution.
> - I will scrape for the purpose of creating new value from the data, not to duplicate it."

<sup>*</sup> We'll explain the meaning of "User Agent" later.

Equipped with this information, we'll now start scraping.

### B.1 First check the copyright

**Exercise 01:** Take a look at the RSPB website [Copyright information](https://www.rspb.org.uk/help/copyright) and [Terms and Conditions](https://www.rspb.org.uk/help/terms--conditions/). Can you see anything that would prevent us scraping the information? (Note that education is classed as a non-commercial activity.) What restrictions are there on how you can use the information?

In [ ]:
# Run this cell to be offered with hints and solution
show(question=1)

Your answer:


### B.2 Requesting robots.txt

A web server is just another computer on the internet that is listening for **requests** from other computers on the internet. When we're browsing the web, our web browsers are making requests to web servers all around the internet. The web browser is one example of a **user agent**. We're now going to use the python `requests` package as a user agent.

The first file we will retrieve is called `robots.txt`.

Every web server should have this file to tell robots (i.e. automated user agents like search engine crawlers that move from page to page on the web) what the website owner is happy for them to request. (Note that the file is purely advisory. Bad bots - malware etc - will ignore this file.)

In [ ]:
url = 'https://www.rspb.org.uk/robots.txt'
r = requests.get(url)
print(r.text)

`r` is a request object. `r.text` contains the contents of the `robots.txt` file.

The first line of the file `User-agent: *` tells us that the next section of the file applies to *all* user agents - that includes us! The next lines tell us the areas of the site that should not be scraped automatically. The RSPB don't want us scraping any pages whose URLs start https://www.rspb.org.uk/search ,
https://www.rspb.org.uk/community/SiteMap.ashx etc. There is more information on the format of the `robots.txt` file [here](https://www.robotstxt.org/robotstxt.html).

Note that the web server will have a record of the "User-agent" in its logs. For example, the request we've just made will be recorded in the web server logs like this:
```
77.99.216.20 - - [20/Feb/2021:18:24:40 +0000] "GET /robots.txt HTTP/1.1" 200 4594 "-" "python-requests/2.24.0"
```
This means that a user agent called "python-requests/2.24.0" has made a request from the IP address 77.99.216.20 for the file `robots.txt`. Following the suggested guidelines on ethical web scraping, it would be polite to advertise who we are, by changing the name of the user agent in the request to something like this:

In [ ]:
headers = {'user-agent': 'Foundations of Data Science Course (https://www.github.com/Inf2-FDS/weekS2-06-web-scraping)'}
r = requests.get(url, headers=headers)

This request will appear in the server logs like this:
```
77.99.216.20 - - [20/Feb/2021:19:57:16 +0000] "GET /robots.txt HTTP/1.1" 200 4594 "-" "Foundations of Data Science Course (https://www.github.com/Inf2-FDS/weekS2-06-web-scraping)"
```
If the site owner gets annoyed by frequent requests, they can at least let us know about the problem. But please, when you web scrape, "request data at a reasonable rate" (as per the ethical guidelines above). I.e. don't make lots of requests in quick succession (see later on how to do this).

Note that the server log also includes the Status code 200 - this means that the request has been delivered successfully by the server. At our end we can check that the code is 200:

In [ ]:
r.status_code

If the code starts with 2, it means the request was successful. If it starts with 3, it means that the request was successful. If it starts with 4, it means that the page doesn't exist, and if it starts with 5, it indicates an error at the server end. For full information on all status codes, see the [Mozilla developer documentation's page on status codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

**Exercise 02:** Make a request to the non-existent page https://www.rspb.org.uk/dodo and report on the error code

In [ ]:
# Run this cell to be offered with hints and solution
show(question=2)

In [ ]:
# Your code

## C. Getting our first bird

**Exercise 03:** Get the RSPB web page for the White-tailed eagle:
https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/white-tailed-eagle/ and print the text of the page that you have retrieved.

In [ ]:
# Run this cell to be offered with hints and solution
show(question=3)

In [ ]:
# Your code

### C.1 Quick review of HTML and some useful terminology

What you should see is the Hypertext Markup Language (HTML) source of the page. It's exactly what you would see if you looked at the page source in the browser.

You probably know already, but an HTML document has a nested structure, like this:
```
<!DOCTYPE html>
<html>
  <head>
    <title>Page title</title>
    <script src='https://www.googletagmanager.com/gtag/js?id=UA-11409245-4'/>
  </head>
  <body>
    <h1>Aquatic warbler</h1>
    <p style='color: red'>Stuff about the aquatic warbler</p>
  </body>
</html>
```
The angle brackets containing text are called [tags](https://developer.mozilla.org/en-US/docs/Glossary/Tag). Tags either:
- come in pairs, with an opening tag and a closing tag, e.g. `<h1>...</h1>`
- or they may be a single tag, in which case they have  slash to indicate that they are a closing tag: e.g. `<script ... />`

Some tags have [attributes](https://developer.mozilla.org/en-US/docs/Glossary/Attribute), such as the `style` attribute in the opening `<p>` tag and the `src` attribute in the `<script>` tag.

*Everything* between a pair of tags is an [element](https://developer.mozilla.org/en-US/docs/Glossary/Element). For example, the `<html>` element is all the text (apart from the `DOCTYPE`). The `<head>` element contains metadata, while the `<body>` element contains text that is visible on the website:
```
<body>
  <h1>Aquatic warbler</h1>
  <p style='color: red'>Stuff about the aquatic warbler</p>
</body>
```

The indentation structure in the code indicates that there is a family tree-like structure to the document. We call the `<html>` the root element. It has two children (`<head>` and `<body`>). In turn `<body>` has two children: `<h1>` and `<p>`. Because `<h1>` and `<p>` have the same parent, they are [siblings](https://dictionary.cambridge.org/us/dictionary/english/sibling).

### C.2 Making BeautifulSoup from the bird page

You can just see some tags in the text returned by our request, but the tree structure is not clear - we just have a string of text. Fortunately there is a Python package called [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) that will help use to _parse_ the text so that we can extract the tags and their contents (i.e. the elements) easily.

**Note:** Crazy name, crazy package. The name BeautifulSoup relates to an episode in an equally crazy book, [*Alice in Wonderland*](http://www.gutenberg.org/ebooks/11) by the mathematician Lewis Carroll, which features a number of birds, including an Eaglet (a baby eagle). Take a minute to look at the video of the White tailed eaglet being fed by its parents on the page we are scraping: https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/white-tailed-eagle/

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')
soup

We can now see that HTML code is printed in a somewhat prettier way. In fact BeatifulSoup has reconstructed the parse tree of the document.

### C.3 Accessing tags and elements

We can access the first tag of a particular type using the following syntax:

In [ ]:
## The title element
print(soup.title)

## The name of the tag
print(soup.title.name)

## The contents of the tag
print(soup.title.string)

## An img (Image) tag
print(soup.img)

## An a (Anchor) tag
print(soup.a)

Note the last tag we have retrieved is an anchor (i.e. link). It has an attribute `href` containing the target, for example a URL. We can extract the contents of the `href` attribute like this:

In [ ]:
soup.img['alt']

**Exercise 04:** Get the contents of the first `h1` tag. (`h1` stands for "Heading text level 1"):

In [ ]:
# Run this cell to be offered with hints and solution
show(question=4)

In [ ]:
## Your code

Hopefully you've now got the name of the bird, with one line of code.

### C.4 Navigating deeper into the tree

Now to find the length, the wingspan, the weight and the number of UK breeding pairs of the White tailed eagle.

We can navigate down the tree structure like this:

In [ ]:
## Find the first `div` element that is the child of the `body` element
soup.body.div

**Exercise 05:** Have a go at navigating the tree by extending the syntax above, e.g.:
```
soup.body.div.div.div.div.div
```

In [ ]:
# Run this cell to be offered with hints and solution
show(question=5)

In [ ]:
# Your code

You can see that it's going to be tricky navigating a tree like this.

### C.5 Finding the measurements

Probably the trickiest part of web scraping, is finding the elements to extract, in this case the measurements. One way to do this is to use the "Inspector" function in your browser. In Firefox you can do this by pressing `Ctrl-Shift-I` - other browsers have similar developer tools. Then, as in the picture below:

1. Select the "element picker"
2. Click on the part of the page you are interested in (we'll call this the element of interest)
3. Look at the corresponding code. You can copy it by right-clicking on it and selecting **Copy->Outer HTML**.

![Screenshot of Inspector in Firefox](src/Screenshot-inspector-annotated.png)

The HTML we have found looks like this:
```
<div class="species-measurements-population__measurements">
    <h3 class="key-info-title-measurements">Measurements:</h3>
    <dl class="species-measurements-population__details">
        <dt class="species-measurements-population__details-label">Length:</dt>
        <dd class="species-measurements-population__details-content">
            13cm
        </dd>
    </dl>

    <dl class="species-measurements-population__details">
        <dt class="species-measurements-population__details-label">Wingspan:</dt>
        <dd class="species-measurements-population__details-content">
            16.5-19.5cm
        </dd>
    </dl>

    <dl class="species-measurements-population__details">
        <dt class="species-measurements-population__details-label">Weight:</dt>
        <dd class="species-measurements-population__details-content">
            10-14g
        </dd>
   </dl>
</div>
```

We now need to work out how to retrieve these elements. Maybe if we could get all of the `<dl>`  ("description list") elements, we would pick up the measurements we want, and no other `<dl>` elements in the document.

### C.6 The `find_all()` method

So far, we've been able to find the first instance of any element in the tree, but we want all three `<dl>` elements. More often than not the element or elements we want won't be the first occurrence in the document. Fortunately BeautifulSoup offers other methods of navigating the tree.  If we want to extract all the `<dl>` elements we can use the `find_all` function like this:

In [ ]:
soup.body.find_all('dl')

This looks more promising. We now have a list of elements that we can iterate over, like this:

In [ ]:
measurements = soup.body.find_all('dl')
for measurement in measurements:
    print(measurement.dt.string)
    print(measurement.dd.string)

However, there is a lot of white space in what we have scraped. How do we get rid of it?

### C. Enter regular expressions

You encountered [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) (also referred to as **regexs**) in the *Data wrangling II* lab. To recap, regexs are sets of character combinations that Python (and many other programming languages) can understand and that are used to search through character strings. We will demonstrate how to use the python `re` package to remove whitespace.

The `re.sub` function replaces the parts of a string matching a pattern with a desired replacement. E.g.:

In [ ]:
re.sub("^\\s+|\\s+$", '', measurements[0].dd.string)

 Let’s break down `^\\s+|\\s+$` to understand what it means:

`^` = From start of string

`\\s` = White space

`+` = Select 1 or more instances of the character before

`|` = “or”

`$` = To the end of the line

So `^\\s+|\\s+$` can be interpreted as "Select one or more white spaces that exist at the start of the string and select one or more white spaces that exist at the end of the string".

## D. Putting it all together

We now know everything we need to know to extract the data from the page and create a pandas DataFrame containing the following information:

- Species of bird
- All measurements associated with the bird

### D.1 Text to DataFrame

**Exercise 06:** Create a function `get_measurements` that takes the text of a bird species page on the RSPB website (i.e. `r.text`, and returns a Pandas DataFrame with the above information. Make sure that any whitespace has been removed. Apply the function on the text above and store the DataFrame as `eagle`.

In [ ]:
# Run this cell to be offered with hints and solution
show(question=6)

In [ ]:
## Your code

### D.2 Joining DataFrames

**Exercise 07:** Use your function to get the corresponding DataFrame for the Aquatic warbler: https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/aquatic-warbler/ . We suggest you store the data frame in a variable `warbler`.

In [ ]:
# Run this cell to be offered with hints and solution
show(question=7)

In [ ]:
## Your code

We notice that the set of measurements for the aquatic warbler is not the same as for the eagle - e.g we don't have the `UK breeding` measurement, but we do have a `Europe` measurement.

**Exercise 08:** Create a unified table from the `eagle` and `warbler` DataFrames. If a measurement doesn't exist for one bird, but does for the other, the non-existent measurement should be set to `NaN`.

In [ ]:
# Run this cell to be offered with hints and solution
show(question=8)

In [ ]:
# Your code

### D.3 Scraping multiple pages

We've now finished the practical part of this lab, but here is how we would proceed if we wanted to automate the scraping of more birds from the RSPB website. To do this we would:
- Get the contents of the A-Z listing: https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/   (The `robots.txt` file says this is OK.)
- Find the links to the pages with birds which we will find by searching for `<a>` tags and extracting the `href` attributes.
- Create an empty data frame
- For each page:
  - Request the text of the page
  - Run the function that we've just written
  - Store the series in the data frame
  - Run `time.sleep(10)` - this means that there is a pause of 10 seconds before we make the next request. This will reduce the load on the RSPB website, and prevent them from thinking that we are trying to mount a DDoS attack. Web servers can be configured to detect rapid requests in succession and can then block further requests.
  
It gets more complicated, because this there are only 16 birds listed on each A-Z page, and you need to find a way of iterating through each listing page. There is no need to undertake this exercise. If you have to do web scraping you will find that every website is structured differently, and you will need to find a strategy that works for that site.

It is also worth remembering that sites change their structure quite often, so if you programmed a scraper for a site last year, it may not work this year.

**Exercise 09: (Optional)** Implement the above procedure for one page worth of birds on the RSPB website.

### D.4 Cleaning the data

Although we have the measurements that we wanted, it will be hard to analyse them numerically, since they are very unstructured. For example:
- We have **Wingspans** of "16.5-19.5cm" and "200-240cm". Perhaps we would like to have two columns for the minimum and maximum wingspan in centimetres?
- We have **Weights**  of "3.5-5kg (male); 4-7kg (female)" and "10-14g". Perhaps we would like to convert this into four columns: minimum/maximum Male/Female weight in grammes? But can we assume that the weight range of the Aquatic warbler applies both to Males and Females?

We can see why data scientists estimate that about 80% of the time spent on a data science project is involved in data cleaning. It is important to record the steps that you have undertaken in data cleaning, and also important to keep a copy of the original, raw, data, so that your cleaning can be reproduced or improved upon.

**Exercise 10: (Optional)** Write down a scheme for how you want to clean the data frame you have collected in Exercise 09. Implement your scheme using regex substitution. Visualise your cleaned data.

## The last FDS lab

This was the last FDS lab. We hope you enjoyed it! If you have any comments on the labs or the organisation of the lab sessions, please [email David](mailto:david.c.sterratt@ed.ac.uk?subject=Labs).

![Disappearing cheshire cat. Public domain, downloaded from https://www.alice-in-wonderland.net](src/alice-in-wonderland-cheshire-cat-vanishing.jpg)
The Cheshire cat disappears in *Alice in Wonderland*.